In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

page = requests.get('https://www.imdb.com/list/ls076655138/')
print(page)
soup = BeautifulSoup(page.content,'html.parser')

<Response [200]>


In [2]:
# scraping site for data on genre, stars(actors) and filming dates

genre_data = soup.find_all('span',class_='genre')
cast_data = soup.find_all('p', class_="text-muted text-small")

directors_and_actors = []   # empty list to store text data from cast
genre = []     # empty list to store text data from genre
director_name = []      # empty list to store director's names in order
stars = []         # empty list to store actor's names in order

for i in range(len(genre_data)):
    genre.append(genre_data[i].text.strip())

for i in range(1,len(cast_data),3):
    directors_and_actors.append(cast_data[i].text.strip())

# Seperating the director's and actor's names from list
for i in range(len(directors_and_actors)):
    empty_string = ''
    for j in range(len(directors_and_actors[i])):
        if ord(directors_and_actors[i][j]) != 124:
            empty_string = empty_string + directors_and_actors[i][j]
        else:
            director_name.append(empty_string.strip('Director:').strip())
            break
    empty_string = ''
    for k in range(1,len(directors_and_actors[i])):
        if ord(directors_and_actors[i][-k]) != 124:
            empty_string = directors_and_actors[i][-k] + empty_string
        else:
            stars.append(empty_string.strip().strip('Stars:').replace('\n',''))
            break

In [3]:
# scraping for movie titles, descriptions and realease dates
div = soup.find_all('div', {'class': 'lister list detail sub-list'})
movie_name = soup.find_all('h3', {'class': 'lister-item-header'})
description = soup.find_all('p', class_='')

movie_name_list = []
description_list = []
year_list = []

for x in range(100):
    movie_name_list.append(movie_name[x].text.strip("").replace("\n",""))
    year_list.append(movie_name[x].text.strip("").replace("\n",""))
    year_list[x] = ''.join([j for j in year_list[x] if j.isdigit()])
    movie_name_list[x] = ''.join([j for j in movie_name_list[x] if not j.isdigit()])
for x in range(len(year_list)):
    year_list[x] = year_list[x][-4:]
    movie_name_list[x] = movie_name_list[x][1:-2]
    description_list.append(description[x].text.strip("\n"))


In [4]:
rating_data = soup.find_all('span', class_="ipl-rating-star__rating")
rating = [] #empty list
for i in range(0,len(rating_data),23): #loop
    
    rating.append(rating_data[i].text)
rating


duration_data = soup.find_all('span', class_="runtime")
duration = [] #empty list

for i in range(len(duration_data)):
    duration.append(duration_data[i].text)


In [7]:
df = pd.DataFrame({
    'Movie':movie_name_list,
    'Description':description_list,
    'Release Date':year_list,
    'Director':director_name,
    'Rating':rating,
    'Duration':duration,
    'Genre':genre,
    'Stars':stars,
})

df.to_csv('top_100_drama.csv')
df

,Movie,Description,Release Date,Director,Rating,Duration,Genre,Stars
0,Stirb langsam,An NYPD officer tries to save his wife and sev...,1988,John McTiernan,8.2,132 min,"Action, Thriller","Bruce Willis, Alan Rickman, Bonnie Bedelia, Re..."
1,Jäger des verlorenen Schatzes,"In 1936, archaeologist and adventurer Indiana ...",1981,Steven Spielberg,8.4,115 min,"Action, Adventure","Harrison Ford, Karen Allen, Paul Freeman, John..."
2,Indiana Jones und der Tempel des Todes,"In 1935, Indiana Jones arrives in India, still...",1984,Steven Spielberg,7.5,118 min,"Action, Adventure","Harrison Ford, Kate Capshaw, Ke Huy Quan, Amri..."
3,Indiana Jones und der letzte Kreuzzug,"In 1938, after his father Professor Henry Jone...",1989,Steven Spielberg,8.2,127 min,"Action, Adventure","Harrison Ford, Sean Connery, Alison Doody, Den..."
4,Terminator : Tag der Abrechnung,"A cyborg, identical to the one who failed to k...",1991,James Cameron,8.5,137 min,"Action, Sci-Fi","Arnold Schwarzenegger, Linda Hamilton, Edward ..."
...,...,...,...,...,...,...,...,...
95,Bad Boys II,Two loose-cannon narcotics cops investigate th...,2003,Michael Bay,6.6,147 min,"Action, Comedy, Crime","Will Smith, Martin Lawrence, Gabrielle Union, ..."
96,Snowpiercer,In a future where a failed climate change expe...,2013,Bong Joon Ho,7.1,126 min,"Action, Drama, Sci-Fi","Chris Evans, Jamie Bell, Tilda Swinton, Ed Harri"
97,Agenten sterben einsam,Allied agents stage a daring raid on a castle ...,1968,Brian G. Hutton,7.6,158 min,"Action, Adventure, War","Richard Burton, Clint Eastwood, Mary Ure, Patr..."
98,Das dreckige Dutzend,"During World War II, a rebellious U.S. Army Ma...",1967,Robert Aldrich,7.7,150 min,"Action, Adventure, War","Lee Marvin, Ernest Borgnine, Charles Bronson, ..."
